In [ ]:
from transformers import AutoTokenizer, GPTJForCausalLM
from tqdm import tqdm
import torch
import pandas as pd

In [ ]:
df = pd.read_csv("data/training_data_100000.csv")

In [ ]:
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6b", torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b", padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
from tqdm import tqdm
batch_size = 16
generated_phrases = []
phrases_index = []

batch_texts = []
batch_idxs = []
for i in tqdm(range(100000)):
#for i in tqdm(range(50000)):
    if len(batch_texts) < batch_size:
        batch_texts.append(str(df.loc[i, "decoded_prefix"]))
        batch_idxs.append(i)
        if i == 99999:
            batch = tokenizer(batch_texts, padding=True, return_tensors="pt", max_length=2048)
            batch = {k: v.cuda() for k, v in batch.items()}
            outputs = model.generate(**batch, max_new_tokens=11)
            generated_phrases.extend(outputs.tolist())
            phrases_index.extend(batch_idxs)
    elif len(batch_texts) == batch_size:
        batch = tokenizer(batch_texts, padding=True, return_tensors="pt", max_length=2048)
        batch = {k: v.cuda() for k, v in batch.items()}
        outputs = model.generate(**batch, max_new_tokens=11)
        generated_phrases.extend(outputs.tolist())
        phrases_index.extend(batch_idxs)
        batch_texts = []
        batch_texts.append(df.loc[i, "decoded_prefix"])
        batch_idxs.append(i)

In [ ]:
from tqdm import tqdm

processed_phrases = []
for i in tqdm(range(len(generated_phrases))):
    decoded_prefix = tokenizer(df.loc[i, "decoded_prefix"], padding=False, max_length=2048)["input_ids"]
    processed_phrase = [tk for tk in generated_phrases[i] if tk != 50400]
    processed_phrase = processed_phrase[len(decoded_prefix):]
    processed_phrase = tokenizer.decode(processed_phrase)
    processed_phrases.append(processed_phrase)

In [ ]:
# df["teacher_phrase"] = processed_phrases
processed_df = df[:len(generated_phrases)]

In [ ]:
processed_df["teacher_phrase"] = processed_phrases
processed_df.to_csv("data/training_data_teacher_100000.csv", index=False)

In [ ]:
df.to_csv("data/training_data_teacher_100000.csv", index=False)

In [ ]:
processed_df.to_csv("data/training_data_teacher_11000.csv", index=False)